In [1]:
# 高德地图路径规划

# 计算图中从start到end的最短距离
# graph, cost开销字典, parents记录路径, processed已经执行过

def dijkstra():
    # 找到目前开销最小的节点
    node = find_lowest_cost_node(costs)
    #print('当前cost最小节点：', node)
    
    # 找到开销最小的节点就进行路径规划，如果所有节点都放到processed中，就结束
    while node is not None:
        # 获取节点目前的cost
        cost = costs[node]
        # 获取节点的邻居
        neighbors = graph[node]
        # 遍历所有邻居，看看是否通过node节点，比之前cost更少
        for neighbor in neighbors.keys():
            # 计算经过当前节点到达相邻节点的开销， 即当前节点cost + 当前节点到邻居的cost
            new_cost = cost + neighbors[neighbor]
            # 通过node是否可以更新start到neighbor的cost  
            if neighbor not in costs or new_cost < costs[neighbor]:
                costs[neighbor] = new_cost
                # 经过node到达neighbor的节点cost更少
                parents[neighbor] = node
        # 将当前节点放入processed
        processed.append(node)
        # 找到接下来要处理的节点， 并且继续循环
        node = find_lowest_cost_node(costs)
        #print('当前cost最小节点：', node)   
    #print(parents)
    # 循环完毕说明所有节点都已经处理完
    shortest_path = find_shortest_path()
    print('从{}到{}的最短路径：{}'.format(start,end,shortest_path))

In [2]:
def find_shortest_path():
    node = end
    shortest_path = [end]
    while parents[node] != start:
        shortest_path.append(parents[node])
        node = parents[node]
    shortest_path.append(start)
    shortest_path.reverse()
    return shortest_path

In [3]:
# 找到开销最小的节点
def find_lowest_cost_node(costs):
    # 初始化数据
    lowest_cost = float('inf')
    lowest_cost_node = None
    # 遍历所有节点
    for node in costs:
        # 找到非processed集合中最小节点
        if not node in processed:
            # 如果当前节点开销比已经存在的开销小，则更新该节点为开销最下的节点
            if costs[node] < lowest_cost:
                lowest_cost = costs[node]
                lowest_cost_node = node
    return lowest_cost_node

In [4]:
# 计算两点之间的距离
import re
key =  'd31ca20f3fc9f31d76e20359404a0783'
import requests
def compute_distance(longitude1, latitude1, longitude2, latitude2):
    request_url = 'http://restapi.amap.com/v3/distance?key='+ key+ '&origins=' + str(longitude1) +','\
                    +str(latitude1) +'&destination='+str(longitude2) +','+str(latitude2)+'&type=1'
    
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    data = requests.get(request_url, headers=headers, timeout=10)
    data.encoding='utf-8'
    data = data.text
    #print(data)
    pattern = 'distance":"(.*?)","duration":"(.*?)"'
    result = re.findall(pattern, data)
    
    return int(result[0][0])
#compute_distance(116.337581,39.993138,116.339941,39.976228)

In [5]:
compute_distance(116.337581,39.993138,116.339941,39.976228)

2646

In [6]:
# 数据加载
import pandas as pd
data = pd.read_csv('./subway.csv')
data

,name,site,city,longitude,latitude
0,四惠站,地铁八通线,北京,116.498489,39.908279
1,四惠东站,地铁八通线,北京,116.514954,39.908051
2,高碑店站,地铁八通线,北京,115.858901,39.328727
3,传媒大学站,地铁八通线,北京,116.554931,39.909548
4,双桥站,地铁八通线,北京,116.569498,39.901066
...,...,...,...,...,...
346,永丰站,地铁16号线,北京,116.237766,40.071247
347,永丰南站,地铁16号线,北京,116.247876,40.064807
348,西北旺站,地铁16号线,北京,116.258460,40.048344
349,马连洼站,地铁16号线,北京,116.274073,40.032358


In [7]:
# graph保存图中的邻接矩阵表（点到邻居之间的距离）
from collections import defaultdict
# 创建图中两点之间的距离
graph = defaultdict(dict)
for i in range(data.shape[0]):
    site1 = data.loc[i,'site']
    if i < data.shape[0]-1:
        site2 = data.loc[i+1, 'site']
        # 如果两个站在同一条地铁线
        if site1==site2:
            longitude1, latitude1 = data.loc[i, 'longitude'], data.loc[i, 'latitude']
            longitude2, latitude2 = data.loc[i+1, 'longitude'], data.loc[i+1, 'latitude']
            name1, name2 = data.loc[i, 'name'], data.loc[i+1, 'name']
            distance = compute_distance(longitude1, latitude1, longitude2, latitude2)
            # 无向图，双向保存
            graph[name1][name2] = distance
            graph[name2][name1] = distance
            print(name1, name2, distance)

四惠站 四惠东站 4138
四惠东站 高碑店站 108548
高碑店站 传媒大学站 111743
传媒大学站 双桥站 4827
双桥站 管庄站 3647
管庄站 八里桥站 3243
八里桥站 通州北苑站 3409
通州北苑站 果园站 1472
果园站 九棵树站 1510
九棵树站 梨园站 2323
梨园站 临河里站 3197
临河里站 土桥站 1674
西二旗站 生命科学园站 6810
生命科学园站 朱辛庄站 3479
朱辛庄站 巩华城站 6679
巩华城站 沙河站 6320
沙河站 沙河高教园站 3845
沙河高教园站 南邵站 6743
南邵站 北邵洼站 1981
北邵洼站 昌平东关站 2741
昌平东关站 昌平站 11188
昌平站 十三陵景区站 9933
十三陵景区站 昌平西山口站 2093
苏庄站 良乡南关站 2180
良乡南关站 良乡大学城西站 2266
良乡大学城西站 良乡大学城站 2708
良乡大学城站 良乡大学城北站 1514
良乡大学城北站 广阳城站 2526
广阳城站 篱笆房站 2344
篱笆房站 长阳站 2237
长阳站 稻田站 4280
稻田站 大葆台站 15847
大葆台站 郭公庄站 1958
宋家庄站 肖村站 3417
肖村站 小红门站 3728
小红门站 旧宫站 4242
旧宫站 亦庄桥站 3397
亦庄桥站 亦庄文化园站 2001
亦庄文化园站 万源街站 2362
万源街站 荣京东街站 1537
荣京东街站 荣昌东街站 1465
荣昌东街站 同济南路站 2340
同济南路站 经海路站 2331
经海路站 次渠南站 3177
次渠南站 次渠站 2002
东直门站 三元桥站 3133
三元桥站 三号航站楼站 21947
三号航站楼站 二号航站楼站 6394
二号航站楼站 三元桥站 19870
三元桥站 东直门站 3312
苹果园站 古城站 4656
古城站 八角游乐园站 3191
八角游乐园站 八宝山站 2190
八宝山站 玉泉路站 3591
玉泉路站 五棵松站 1734
五棵松站 万寿路站 2346
万寿路站 公主坟站 2217
公主坟站 军事博物馆站 4645
军事博物馆站 木樨地站 4988
木樨地站 南礼士路站 2051
南礼士路站 复兴门站 1819
复兴门站 西单站 4612
西单站 天安门西站 5112
天安门西站 天安门东

In [8]:
graph

defaultdict(dict,
            {'四惠站': {'四惠东站': 4138, '大望路站': 4415},
             '四惠东站': {'四惠站': 4138, '高碑店站': 108548},
             '高碑店站': {'四惠东站': 108548, '传媒大学站': 111743},
             '传媒大学站': {'高碑店站': 111743, '双桥站': 4827},
             '双桥站': {'传媒大学站': 4827, '管庄站': 3647},
             '管庄站': {'双桥站': 3647, '八里桥站': 3243},
             '八里桥站': {'管庄站': 3243, '通州北苑站': 3409},
             '通州北苑站': {'八里桥站': 3409, '果园站': 1472},
             '果园站': {'通州北苑站': 1472, '九棵树站': 1510},
             '九棵树站': {'果园站': 1510, '梨园站': 2323},
             '梨园站': {'九棵树站': 2323, '临河里站': 3197},
             '临河里站': {'梨园站': 3197, '土桥站': 1674},
             '土桥站': {'临河里站': 1674},
             '西二旗站': {'生命科学园站': 6810, '上地站': 2870, '龙泽站': 6891},
             '生命科学园站': {'西二旗站': 6810, '朱辛庄站': 3479},
             '朱辛庄站': {'生命科学园站': 3479, '巩华城站': 6679, '育知路站': 2964},
             '巩华城站': {'朱辛庄站': 6679, '沙河站': 6320},
             '沙河站': {'巩华城站': 6320, '沙河高教园站': 3845},
             '沙河高教园站': {'沙河站': 3845, '南邵站': 6743

In [9]:
import pickle
output = open('graph.pkl', 'wb')
pickle.dump(graph, output)

In [10]:
start = '五道口站'
end = '传媒大学站'
# 查看start的邻居节点
graph[start].keys()
graph[start].values()

dict_values([2893, 6867])

In [11]:
# 创建节点的开销表， cost是指从start到这个节点的距离
costs = {}
# 存储父节点的Hash表， 用于记录路径
parents = {}
parents[end] = None

# 获取该节点相邻节点
for node in graph[start].keys():
    costs[node] = float(graph[start][node])
    parents[node] = start
costs

{'知春路站': 2893.0, '上地站': 6867.0}

In [12]:
costs[end] = float('inf')
# 记录处理过的节点list
processed = []
dijkstra()

从五道口站到传媒大学站的最短路径：['五道口站', '知春路站', '大钟寺站', '西直门站', '新街口站', '平安里站', '北海北站', '南锣鼓巷站', '东四站', '朝阳门站', '东大桥站', '呼家楼站', '金台路站', '红庙站', '大望路站', '四惠站', '四惠东站', '高碑店站', '传媒大学站']
